(Optional) PyTorch Introduction
================
 
<div class="alert alert-info">
    <strong>Note:</strong> This exercise is optional and only serves as an introduction and cheatsheet to the general concepts of PyTorch.
</div>

PyTorch is a scientific computing package for Python:

-  Tensor and Neural Network computations (in particular deep learning)
-  Research oriented (in comparison to e.g. TensorFlow)
-  Dynamic computational graph (in comparison to e.g. TensorFlow)
-  “NumPy on the GPU”
-  Backend and API heavily inspired by the original Torch written in Lua

An in-depth tutorial of the concepts described in this notebook can be found [here](https://github.com/jcjohnson/pytorch-examples).

Use the [pytorch website](https://pytorch.org/) and install the newest version.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/My Drive/i2dl/PytorchIntroAndOptionalCNN

/content/drive/My Drive/i2dl/PytorchIntroAndOptionalCNN


In [4]:
%matplotlib inline
import numpy as np
import torch

print(torch.__version__)  # This should print 1.0.x

1.3.1


Tensors
=====

The PyTorch `Tensor` class is very similar to the NumPy `ndarray` class. Their main distinction is the ability of PyTorch Tensors to be used on a GPU which lets them benefit from vastly accelerated and parallelized computations. In order to work with PyTorch it is crucial to understand the basic behavior of its `Tensor` class.

Let's start with the initialization of a regular `5x3` matrix `Tensor`:

In [5]:
x = torch.Tensor(5, 3)
print(x)

tensor([[9.2858e-36, 0.0000e+00, 3.7835e-44],
        [0.0000e+00,        nan, 0.0000e+00],
        [1.3733e-14, 6.4069e+02, 4.3066e+21],
        [1.1824e+22, 4.3066e+21, 6.3828e+28],
        [3.8016e-39, 2.1749e+23, 0.0000e+00]])


The same matrix can be initialized with random entries:



In [6]:
x = torch.rand(5, 3)
print(x)

tensor([[0.9039, 0.7340, 0.7250],
        [0.4571, 0.5199, 0.7768],
        [0.3266, 0.6783, 0.1148],
        [0.7041, 0.0210, 0.2536],
        [0.5103, 0.7978, 0.1930]])


The size of a `Tensor` can be retrieved with:



In [7]:
print(x.size())

torch.Size([5, 3])


<div class="alert alert-info">
    <h3>Note</h3>
    <p>In contrast to a static computational graph of for example Tensorflow the dynamic graph of PyTorch allows to retrieve information such as its size at any time during runtime.</p>
</div>

Tensor Operations
--------

There are multiple syntaxes for `Tensor` operations. We illustrate the different options on the example of `Tensor` addition.

Regular (NumPy) syntax:



In [8]:
y = torch.rand(5, 3)
print(x + y)

tensor([[0.9934, 1.4217, 1.0490],
        [0.8860, 0.7140, 1.0905],
        [1.2260, 1.5080, 0.2741],
        [0.9692, 0.1482, 1.0388],
        [0.5426, 0.9298, 0.2042]])


PyTorch syntax:



In [9]:
print(torch.add(x, y))

tensor([[0.9934, 1.4217, 1.0490],
        [0.8860, 0.7140, 1.0905],
        [1.2260, 1.5080, 0.2741],
        [0.9692, 0.1482, 1.0388],
        [0.5426, 0.9298, 0.2042]])


PyTorch syntax with specific output variable:



In [10]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)


tensor([[0.9934, 1.4217, 1.0490],
        [0.8860, 0.7140, 1.0905],
        [1.2260, 1.5080, 0.2741],
        [0.9692, 0.1482, 1.0388],
        [0.5426, 0.9298, 0.2042]])


PyTorch syntax for inplace operations:

In [11]:
# adds x to y
y.add_(x)
print(y)

tensor([[0.9934, 1.4217, 1.0490],
        [0.8860, 0.7140, 1.0905],
        [1.2260, 1.5080, 0.2741],
        [0.9692, 0.1482, 1.0388],
        [0.5426, 0.9298, 0.2042]])


<div class="alert alert-info">
    <h3>Note</h3>
    <p>Any operation that mutates a <code>Tensor</code> in-place is post-fixed with an <code>_</code>.</p>
    <p>For example: <code>x.t_()</code> (transposing x), <code>x.copy_(y)</code> (copy y to x).</p>
</div>

`Tensor` indexing works just like standard NumPy indexing. And since recently PyTorch even supports `Tensor` [broadcasting](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html)!



In [12]:
print(x[:, 0])

tensor([0.9039, 0.4571, 0.3266, 0.7041, 0.5103])


NumPy: There and back again
---------------------------

Converting a PyTorch `Tensor` to a NumPy `ndarray` and vice versa is a very simple. The `Tensor` and the `ndarray` will share the location of the underlying memory, and changing one will also change the other.

Converting a `Tensor` to a `ndarray` works by simply calling the `Tensor.numpy()` method:

In [13]:
a = torch.ones(5)
b = a.numpy()
print(a)
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


Changing the `Tensor` effects the `ndarray` as well:

In [14]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


The conversion from a `ndarray` to a `Tensor` is just as simple and holds the same properties:

In [15]:
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


Every `Tensor` allocated on the CPU (except the `torch.CharTensor`) support converting to
NumPy and back.

Tensors on the GPU
------------------

PyTorch Tensors can be moved onto a GPU using the ``Tensor.to()`` method. Before converting a GPU `Tensor` to NumPy it has to be moved back to the CPU by calling the ``Tensor.to()`` method again.



In [16]:
# first check if cuda is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda:0"):
    x = x.to(device)
    y = y.to(device)
    z = x + y
    
    print(z)
    print(z.cpu().detach().numpy())
else:
    print("CUDA not available.")

tensor([[1.8973, 2.1557, 1.7740],
        [1.3431, 1.2339, 1.8674],
        [1.5525, 2.1863, 0.3889],
        [1.6733, 0.1691, 1.2924],
        [1.0528, 1.7276, 0.3972]], device='cuda:0')
[[1.8972931  2.155666   1.7739854 ]
 [1.3430641  1.2338556  1.8673608 ]
 [1.5525291  2.186296   0.3888865 ]
 [1.6733133  0.16913372 1.2923894 ]
 [1.0528302  1.7276345  0.39716297]]


More on PyTorch Tensors
-----------------------

The documentation of many more `Tensor` operations, including transposing, indexing, slicing, mathematical operations, linear algebra, random numbers can be found [here](http://pytorch.org/docs/torch).


Autograd - automatic differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package. The package provides automatic differentiation for all operations on Tensors. PyTorch is a define-by-run framework, which means that the calculation of gradients ( e.g. during backpropagation) is defined at runtime and can be different at every single iteration.

Since pytorch 0.4.0, the Tensor class includes the Variable class, and supports nearly all of its operations. Once a computational graph for a Tensor that requires gradients is executed the ``Tensor.backward()`` method can be used to automatically compute all the gradients.

If the ``Tensor`` is not a scalar, the ``backward()`` method requires an additional ``grad_output`` argument which matches the shape of the ``Tensor``. ``grad_output`` is supposed to be the gradient w.r.t the given output. For a scalar ``Tensor`` ``grad_output`` is assumed to be `torch.Tensor([1.0])`.

The `autograd` package additionally provides a `Function` class which encodes a complete history of computation. Each `Tensor` with the `required_grad` attribute has a ``Tensor.grad_fn`` attribute which references the ``Function`` (e.g. an operation such as addition) that created the respective ``Tensor`` and thereby determines its gradient. For Tensors that were created by the user and not as a result of an operation the ``grad_fn`` attribute is ``None``.

The following simple examples will illustrate the basic concepts of the ``autograd`` package.

In [17]:
# In general, tensors don't track gradients
x = torch.ones(1)
x.requires_grad

False

In [18]:
# Thus we can't call the backward function on these tensors
try:
    x.backwards()
except AttributeError:
    print("Doesn't work...")

Doesn't work...


In [19]:
# Enable gradient tracking
x = torch.ones((2, 2), requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Apply an operation to the `Tensor`:



In [20]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


Since ``y`` was created as a result of an operation it has a ``grad_fn`` attribute (`Function`) unequal to `None`:



In [21]:
print(y.grad_fn)

Applying more operations to `y` increases the computational graph:



In [22]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


Gradients
---------
The gradient w.r.t the input `x` can now be computed (backpropagated) with ``out.backward()``. Remember for a scalar this is equivalent to doing ``out.backward(torch.Tensor([1.0]))``.



In [0]:
out.backward()

The input `x` was a `2x2` `Tensor` and therefore $\frac{d(out)}{dx}$ yields a matrix with the same shape:



In [24]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


For such a small computation graph the solution can easily be verified:

The output w.r.t. the input is given as 
$$
\begin{align}
    out =& \frac{1}{4}\sum_i z_i \\
        =& \frac{1}{4}\sum_i 3y_i y_i \\
        =& \frac{1}{4}\sum_i 3(x_i+2)^2
\end{align}
$$.

Therefore the gradient is $\frac{\partial out}{\partial x_i} = \frac{3}{2}(x_i+2)$, which yields
$\frac{\partial out}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$ for a particular input $x_i=1$.



The `autograd` package in combination with the dynamic graph structure allow to do crazy things such as:



In [25]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.norm() < 1000:
    y = y * 2

print(y)

tensor([-1023.4800,    29.7026,   187.2494], grad_fn=<MulBackward0>)



Neural Networks
===============

The `Tensor` class in combination with the `autograd` package build the foundation for constructing Neural Networks (NNs) with PyTorch. To further fascilitate the construction and training of a NN the ``torch.nn`` package, which depends on `autograd` to define NN models and differentiate them, includes additional NN-specifc classes and helper functions.

For example the ``nn.Module`` class which works as a boilerplate NN model class and eventually contains all the individual layers and the ``Module.forward(x)`` method that infers the input ``x`` and returns the output of a NN.

The following is a grapical illustration of the infamous *LeNet* NN from Yann LeCun. This NN was trained to classify the MNIST dataset of handwritten digit images:

It is a simple feed-forward network which takes the input, feeds it through several layers one after the other, and then finally produces the classification output.


Define *LeNet* with PyTorch
--------------------------

The following is an example implementation of the classification network above: 



In [26]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class LeNet(nn.Module):

    def __init__(self):
        """
        Class constructor which preinitializes NN layers with trainable
        parameters.
        """
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """
        Forwards the input x through each of the NN layers and outputs the result.
        """
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # An efficient transition from spatial conv layers to flat 1D fully 
        # connected layers is achieved by only changing the "view" on the
        # underlying data and memory structure.
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        """
        Computes the number of features if the spatial input x is transformed
        to a 1D flat input.
        """
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Due to the `autograd` package a NN merely requires the definition of the ``Module.forward()`` method. The ``.backward()`` function (which backpropagtes the gradients) is automatically defined. Any `Tensor` operation is allowed in the ``forward`` function.

The learnable parameters of a model are returned by ``Module.parameters()``:



In [27]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Check input and output

In [28]:
x = torch.randn((1, 1, 32, 32))
output = net(x)
print(output)

tensor([[ 0.0904,  0.1222, -0.0414, -0.0550,  0.0796,  0.0053,  0.0827, -0.0661,
         -0.0681,  0.0977]], grad_fn=<AddmmBackward>)


Before backpropagating for example a random gradient, the gradient buffers of all parameters should be set to zero:



In [0]:
net.zero_grad()
output.backward(torch.randn(1, 10))

<div class="alert alert-info">
    <h3>Note</h3>
    <p>Calling the <code>Tensor.backward()</code> method a second time before new inputs are forwarded will through an error. This is due to PyTorch deleting all the intermediary results in order to reduce memory consumption. Calling the <code>.backward()</code> method with the <code>retain_graph=True</code> argument keeps those results.
    </p>
</div>

<div class="alert alert-info">
    <h3>Note</h3>
    <p>The entire <code>torch.nn</code> package only supports inputs that are a mini-batch of samples, and not a single sample. For example, <code>nn.Conv2d</code> will take in a 4D Tensor of <code>nSamples x nChannels x Height x Width</code>. If you have a single sample, just use <code>x.unsqueeze(0)</code> to add a fake batch dimension.
    </p>
</div>

Loss Function
-------------
A loss function takes the (output, target) pair as inputs, and computes a value that estimates "how far away" the output is from the target.

There are several different loss functions predefined under the `torch.nn` package. An example of a simple loss is the ``nn.MSELoss`` which computes the mean-squared error between the input and the target value.

More examples of predefined losses are documented [here](http://pytorch.org/docs/nn.html#loss-functions).

A MSE loss example:

In [30]:
output = net(x)
target = torch.arange(1, 11).unsqueeze(0).float()  # a dummy target with 10 classes
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.3391, grad_fn=<MseLossBackward>)


When ``loss.backward()`` is called, the whole graph is differentiated w.r.t. the loss, and all Tensors with gradients in the graph will have their ``Tensor.grad`` attribute accumulated with the gradient.

Backpropagate the Loss
--------------------

A curical step for optimizing the network weights is the backpropogation of the loss. The nature of a computational graph makes this as easy as calling ``loss.backward()``. But since the gradients will be accumulated to already existing gradients one has to clear them first.

In [31]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0219,  0.0505, -0.0817, -0.0329,  0.0181, -0.0577])


Weights Optimization
------------------
The simplest update rule used in practice for optimizing the weights of a NN is the Stochastic Gradient Descent (SGD):

``weight = weight - learning_rate * gradient``

Like any other NN component the optimization step can be implemented with the basic PyTorch classes.

For example:

In [0]:
def sgd_step(net):
    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, the PyTorch framework contains a small optimization package called ``torch.optim``. It includes various predefined update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>Common optimization options such as the L2-regularization (see <code>weight_decay</code> argument) are already included in the predefined optimization schemes.</p>
</div>

Using it is very simple:

In [0]:
import torch.optim as optim

# create an optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-3)

# a single step of an example training loop
optimizer.zero_grad()   # zero the gradient buffers
output = net(x)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update based on the accumalted gradients

Recap
==============

  -  ``torch.Tensor`` - A multi-dimensional array with a `requires_grad` option to record the history of operations applied to it.
  -  ``nn.Module`` - Neural network module. Convenient way of
     encapsulating parameters, with helpers for moving them to GPU,
     exporting, loading, etc.
  -  ``nn.Parameter`` - A kind of `Tensor`, that is automatically
     registered as a parameter when assigned as an attribute to a
     ``Module``.
  -  ``autograd.Function`` - Implements forward and backward definitions
     of an autograd operation. Every ``Tensor`` operation that requires gradients, creates at
     least a single ``Function`` node, that connects to functions that
     created a ``Tensor`` and encodes its history.

<div class="alert alert-info">
    <h3>Note</h3>
    <p>The `torchvision` package includes many predefined helper funcitons specifally designed for solving computer vision problems.</p>
</div>